In [1]:
from envs.discrete_MDP import DiscreteMDP
from policy_iteration import DiscretePolicy, policy_evaluation
import numpy as np

In [2]:
# creating 4x4 gridworld
# actions: 0 - up, 1 - right, 2 - down, 3 - left

p0 = np.ones(16) / 16
p = np.zeros([16, 16, 4]) # 0 and 16 terminal states
for s in range(1, 15):
    if s - 4 < 0:
        p[s, s, 0] = 1
    else:
        p[s - 4, s, 0] = 1

    if s % 4 == 3:
        p[s, s, 1] = 1
    else:
        p[s + 1, s, 1] = 1

    if s + 4 > 15:
        p[s, s, 2] = 1
    else:
        p[s + 4, s, 2] = 1

    if s % 4 == 0:
        p[s, s, 3] = 1
    else:
        p[s - 1, s, 3] = 1

r = -np.ones([16, 16, 4]) # -1 for all transitions

grid_env = DiscreteMDP(p0, p, r)


In [3]:
p[:, 1, :]

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [4]:
# create policy
policy = DiscretePolicy(grid_env)

In [13]:
policy_evaluation(policy, grid_env, 0.001).round(1)

array([  0., -14., -20., -22., -14., -18., -20., -20., -20., -20., -18.,
       -14., -22., -20., -14.,   0.])